# Задание

20.	 В столовой самообслуживания имеется один раздаточный пункт, на котором отпускаются как первые, так и вторые блюда. Поток посетителей столовой — простейший с интенсивностью $ x $; на отпуск как первого, так и второго блюда идет случайное время, распределенное по показательному закону с одним и тем же параметром $ р $. Некоторые посетители берут и первое, и второе (доля таких посетителей равна $ q $), другие — только второе (доля 1 — $ q $). Найти: 1) условия, при которых существует устойчивый, стационарный режим работы столовой; 2) среднюю длину очереди и среднее время пребывания посетителей в столовой, если посетитель съедает одно блюдо в среднем за время $ t $, а два блюда — за время 2 $ t $. 

Устойчивость — свойство системы сохранять состояние равновесия или некоторого движения при воздействии на нее факторов, вызывающих определенные начальные отклонения.


In [24]:
import numpy as np
import simpy as sp
import statistics as st
import random

# СМО
Системой массового обслуживания (СМО) называется любая система для
выполнения заявок, поступающих в неё в случайные моменты времени. 

$ x $ - интенсивность потока посетителей \
$ p $ - параметр для показательного распределения потока обслуживания \
$ q $ - доля посетителей которые берут 2 блюда \

Класс который предоставляет типы посетителей:

In [25]:
import enum

class VisitorsTypes(enum.Enum):
    one_dish = 1
    two_dish = 2

Класс СМО:

In [26]:
class QueueSystem(object):
    def __init__(self, env, x, p, q):
        self.env = env

        self.x = x
        self.p = p
        self.q = q
        self.resources = sp.Resource(env, 1)

        self.serve_times = []
        self.queue_times = []
        self.queue_len = []

    def start(self, action):
        while True:
            yield self.env.timeout(1 / self.x)
            rand = random.random()
            visitor_type = VisitorsTypes.two_dish if rand <= self.q else VisitorsTypes.one_dish
            self.env.process(action(self, visitor_type))

    def serve(self, visitor_type):
        yield self.env.timeout(np.random.exponential(1.0 / self.p))
        if visitor_type == VisitorsTypes.two_dish:
            yield self.env.timeout(np.random.exponential(1.0 / self.p))

Функция которая реализует процес обработки заявок:

In [27]:
def serve(queue_system, visitor_type):
  with queue_system.resources.request() as request:
      queue_system.queue_len.append(len(queue_system.resources.queue))
      start = queue_system.env.now
      yield request
      queue_system.queue_times.append(queue_system.env.now - start)
      yield queue_system.env.process(queue_system.serve(visitor_type))
      queue_system.serve_times.append((queue_system.env.now - start, visitor_type))

# Подсчет результатов работы СМО

In [28]:
def get_avg_time_in_canteen(queue_system, t):
    all_time_in_canteen = []
    for time, visitor_type in queue_system.serve_times:
        all_time_in_canteen.append(time + visitor_type.value * t)
    return st.mean(all_time_in_canteen)

In [29]:
def get_avg_queue_len(queue_system):
    return st.mean(queue_system.queue_len)

# Тест кейсы

In [58]:
def run_qs(x, p, q, t, work_time):
    env = sp.Environment()
    queue_system = QueueSystem(env, x, p, q)
    env.process(queue_system.start(serve))
    env.run(until = work_time)
    print("Средняя длина очереди: " + str(get_avg_queue_len(queue_system)))
    print("Среднее время пребывания посетителей в столовой: " + str(get_avg_time_in_canteen(queue_system, t)))

In [59]:
print("Тест 1")
run_qs(60, 100, 0.2, 100, 10)

Тест 1
Средняя длина очереди: 0.676126878130217
Среднее время пребывания посетителей в столовой: 117.43906236365484


In [60]:
print("Тест 2")
run_qs(40, 80, 0.4, 200, 10)

Тест 2
Средняя длина очереди: 0.656641604010025
Среднее время пребывания посетителей в столовой: 266.3591838019813
